In [ ]:
library(dplyr)
library(plyr)
library(data.table)
library(readr)
library(magrittr)

# Import the output files from RNA Mutect
data_all <- list.files(path = "~/R/ltrc_rna_samples_dir",  # Identify all patient files
                       pattern = "*.maf.annotated", full.names = TRUE) %>% 
  lapply(read_table) %>%                              # Store all files in list
  bind_rows                                         # Combine data sets into one data set 
data_all     
#save data
write.table(data_all,"~/R/LTRC_data/total_rna_data") # 21,363,023 variants


## filter tumor fractions to 10-30%
#import above data
total_rna_variants <- read.table("~/R/LTRC_data/total_rna_data", header = TRUE)
filtered_rna_variants <- total_rna_variants[total_rna_variants$tumor_f > 0.1 & total_rna_variants$tumor_f < 0.3, ]
#save data
write.table(filtered_rna_variants,"~/R/LTRC_data/filtered_rna_data") # 1,181,035 variants, 1346 patients


#setting unique key function to exclude any duplicate variants
#import above data
filtered_rna_variants <- read.table("~/R/LTRC_data/filtered_rna_data", header = TRUE)
unique_rna_variants <- filtered_rna_variants[!duplicated(filtered_rna_variants[2:7]),]
write.table(unique_rna_variants,"~/R/LTRC_data/unique_rna_variants") #134,540 variants



In [ ]:
### Convert file format to maf for future maf22vcf conversion
```{r}

filtered_rna_variants <- read.table("~/R/LTRC_data/filtered_rna_data", header = TRUE)
#
colnames(filtered_rna_variants)[1] <- "NCBI_Build"
filtered_rna_variants$NCBI_Build <- "GRCh38"
#
colnames(filtered_rna_variants)[2] <- "Chromosome"
filtered_rna_variants$Chromosome <- sub("chr", "", as.character(filtered_rna_variants$Chromosome))
#
colnames(filtered_rna_variants)[8] <- "Tumor_Sample_Barcode"
filtered_rna_variants$Tumor_Sample_Barcode <- "Tumor"
#
colnames(filtered_rna_variants)[3] <- "Start_Position"
#
colnames(filtered_rna_variants)[4] <- "End_Position"
#
colnames(filtered_rna_variants)[5] <- "Reference_Allele"
#
colnames(filtered_rna_variants)[6] <- "Tumor_Seq_Allele1"
#
colnames(filtered_rna_variants)[7] <- "Tumor_Seq_Allele2"
#
filtered_rna_variants$Entrez_Gene_Id <- 0
filtered_rna_variants$Center <- "."
filtered_rna_variants$Strand <- "."
filtered_rna_variants$Variant_Classification <- "."
filtered_rna_variants$Variant_Type <- "."
filtered_rna_variants$dbSNP_RS <- "."
filtered_rna_variants$dbSNP_Val_Status <- "."
filtered_rna_variants$Hugo_Symbol <- ""
filtered_rna_variants$dbSNP_RS <- "."
filtered_rna_variants$dbSNP_Val_Status <- "."
#
filtered_rna_variants$Matched_Norm_Sample_Barcode <- ""
filtered_rna_variants$Match_Norm_Seq_Allele1 <- ""
filtered_rna_variants$Match_Norm_Seq_Allele2 <- ""
filtered_rna_variants$Tumor_Validation_Allele1 <- ""
filtered_rna_variants$Tumor_Validation_Allele2 <- ""
filtered_rna_variants$Match_Norm_Validation_Allele1 <- ""
filtered_rna_variants$Match_Norm_Validation_Allele2 <- ""
filtered_rna_variants$Verification_Status <- ""
filtered_rna_variants$Validation_Status <- ""
filtered_rna_variants$Mutation_Status <- ""
filtered_rna_variants$Sequencing_Phase <- ""
filtered_rna_variants$Sequence_Source <- ""
filtered_rna_variants$Validation_Method <- ""
filtered_rna_variants$Score <- ""
filtered_rna_variants$BAM_File <- ""
filtered_rna_variants$Sequencer <- ""
filtered_rna_variants$Tumor_Sample_UUID <- ""
filtered_rna_variants$Matched_Norm_Sample_UUID <- ""
#
filtered_rna_variants <- filtered_rna_variants[, c("Hugo_Symbol", "Entrez_Gene_Id", "Center", "NCBI_Build","Chromosome", "Start_Position", "End_Position", "Strand", "Variant_Classification", "Variant_Type", "Reference_Allele", "Tumor_Seq_Allele1", "Tumor_Seq_Allele2", "dbSNP_RS", "dbSNP_Val_Status", "Tumor_Sample_Barcode", "Matched_Norm_Sample_Barcode", "Match_Norm_Seq_Allele1", "Match_Norm_Seq_Allele2", "Tumor_Validation_Allele1", "Tumor_Validation_Allele2", "Match_Norm_Validation_Allele1", "Match_Norm_Validation_Allele2", "Verification_Status", "Validation_Status", "Mutation_Status", "Sequencing_Phase", "Sequence_Source", "Validation_Method", "Score", "BAM_File", "Sequencer", "Tumor_Sample_UUID", "Matched_Norm_Sample_UUID", "tumor_f", "t_ref_count", "t_alt_count", "init_t_lod", "t_lod_fstar")]
write.table(filtered_rna_variants, file = "~/R/LTRC_data/filtered_rna_variants.maf", sep = "\t", quote=FALSE, row.names=FALSE )


#filtered_rna_variants_maf <- read.delim("~/R/LTRC_data/filtered_rna_variants.maf", sep = "\t", header = T)

In [ ]:
### Annotate Datasets
#This section requires access to a Linux terminal (Terra terminal)


#Install Ensembl's VEP:
#https://useast.ensembl.org/info/docs/tools/vep/script/vep_download.html#installer

#if installing dependencies (GRCh38) is not successful, do it manually:
# VEP's offline cache for GRCh38
mkdir -p $HOME/.vep/homo_sapiens/102_GRCh38/
rsync -avr --progress rsync://ftp.ensembl.org/ensembl/pub/release-106/variation/indexed_vep_cache/homo_sapiens_vep_106_GRCh38.tar.gz $HOME/.vep/
tar -zxf $HOME/.vep/homo_sapiens_vep_106_GRCh38.tar.gz -C $HOME/.vep/

# reference FASTA file
rsync -avr --progress rsync://ftp.ensembl.org/ensembl/pub/release-106/fasta/homo_sapiens/dna_index/ $HOME/.vep/homo_sapiens/106_GRCh38/

# if FTP site is not working, try: 
curl -O ftp://ftp.ensembl.org/pub/release-98/variation/indexed_vep_cache/homo_sapiens_vep_98_GRCh38.tar.gz # or any link you need from above



#conversion maf2vcf
#1. perl -lne 's/\r//; print "$_";' samples.maf > samples_fixed.maf (Needed to run if CR uses line breaks)
#2. perl maf2vcf_1.pl --input-maf unique_fixed.maf --output-dir ~/snpEff/vcf_files/ --ref-fasta ~/.vep/homo_sapiens/105_GRCh38/Homo_sapiens.GRCh38.dna.toplevel.fa.gz


#Annotate using SNPsift
#java -jar SnpSift.jar annotate hg38 variants.vcf > variants_annotated.vcf

In [ ]:
##Make annotated data more presentable 

library(data.table)
filtered_ann_variants <- fread("~/R/LTRC_data/filtered_rna_rna_variants_annotated.vcf")


#separate INFO section to extrac Gene info
library(dplyr)
gene <- data.frame(filtered_ann_variants[[8]]%>%strsplit("|", fixed=T)%>%sapply("[", 4))
mutation <- data.frame(filtered_ann_variants[[8]]%>%strsplit("|", fixed=T)%>%sapply("[", 2))

variants <- cbind(filtered_ann_variants,gene,mutation)
colnames(variants)[12] <- "GENE"
colnames(variants)[13] <- "MUTATION"
variants$NORMAL <- NULL

#rearrange columns
variants <- variants[, c(1:7,11,12,8:10)]
write.table(variants, file = "~/R/LTRC_data/variants", sep = "\t", quote=FALSE, row.names=FALSE )


#dataset needed to match the patient ID
filtered_test <- fread("~/R/LTRC_data/filtered_test.maf")
test <- filtered_test[ , c(5,6,11,13,16,35,36,37,38,39)]
colnames(test)[1] <- "#CHROM"
colnames(test)[2] <- "POS"
colnames(test)[3] <- "REF"
colnames(test)[4] <- "ALT"
colnames(test)[5] <- "Patient_ID"
write.table(test, file = "~/R/LTRC_data/patient_reference_file", sep = "\t", quote=FALSE, row.names=FALSE )


#merge variants and patient reference files
variants <- read.delim("~/R/LTRC_data/variants", sep = "\t", header = T)
patient_reference_file <- read.delim("~/R/LTRC_data/patient_reference_file", sep = "\t", header = T)
#library(dplyr)
#count <- patient_reference_file%>%count("Patient_ID")
#count1 <- test%>%count("Patient_ID")

filtered_variants_final <- patient_reference_file %>% left_join(variants)
write.table(filtered_variants_final, file = "~/R/LTRC_data/filtered_variants_final", sep = "\t", quote=FALSE, row.names=FALSE )



In [ ]:
## filter out variants occurring in more than 10 individuals
#import data
filtered_variants_final <- read.delim("~/R/LTRC_data/filtered_variants_final", sep = "\t", header = T)
library(dplyr)
variants_filtered_10 <- filtered_variants_final %>%
  group_by(X.CHROM,POS,REF,ALT) %>%
  distinct(.keep_all = TRUE) %>%
  filter(n() <= 10)
write.table(variants_filtered_10,"~/R/LTRC_data/variants_filtered_10", sep = "\t", quote=FALSE, row.names=FALSE ) # 284,997 variants; 1346 patients
#file
variants_filtered_10 <- read.delim("~/R/LTRC_data/variants_filtered_10", sep = "\t", header = T)



#double check count
#library(plyr)
#count <- ddply(variants_filtered_10,.(X.CHROM,POS,REF,ALT),nrow)
#count <- variants_filtered_10%>%count("Patient_ID")

